In [1]:
import requests
from pydriller import Repository
import pandas as pd
from datetime import datetime, timedelta
import re

# Specify the GitHub repository owner and name for Kubernetes
repository_owner = 'kubernetes'
repository_name = 'kubernetes'

# GitHub API endpoint for issues
issues_api_url = f'https://api.github.com/repos/{repository_owner}/{repository_name}/issues'

# Set the start date as the current date minus 24 months
start_date = datetime.now() - timedelta(days=24 * 365 / 12)

# Initialize an empty list to store commit and issue data
commit_data = []
issue_data = []

# Use PyDriller to analyze the Kubernetes repository for commits within the last 24 months
repository_url = f'https://github.com/{repository_owner}/{repository_name}.git'
for commit in Repository(repository_url, since=start_date).traverse_commits():
    commit_info = {
        'commit_id': commit.hash,
        'commit_time': commit.committer_date,
        'committer_id': commit.committer.name,
        'changed_files': len(commit.modified_files),
        'changed_LOC': commit.lines
    }
    commit_data.append(commit_info)

    # Check if the commit message contains issue references
    issue_references = re.findall(r'#(\d+)', commit.msg)
    for issue_id in issue_references:
        # Fetch additional information about the issue using GitHub API
        issue_url = f'{issues_api_url}/{issue_id}'
        issue_response = requests.get(issue_url)
        if issue_response.status_code == 200:
            issue_info = {
                'issue_id': issue_id,
                'issue_content': issue_response.json().get('title', ''),
                'issue_type': issue_response.json().get('labels', [])[0]['name'] if issue_response.json().get('labels') else '',
                'issue_status': issue_response.json().get('state', ''),
                'issue_open_time': issue_response.json().get('created_at', ''),
                'issue_close_time': issue_response.json().get('closed_at', ''),
                'issue_creator': issue_response.json().get('user', {}).get('login', ''),
                'issue_closer': issue_response.json().get('closed_by', {}).get('login', '') if issue_response.json().get('closed_by') else ''
            }
            issue_data.append(issue_info)

# Create Pandas DataFrames from the commit and issue data
commit_df = pd.DataFrame(commit_data)
issue_df = pd.DataFrame(issue_data)

# Clean the data (if needed)
# Add your cleaning operations here

# Save commit and issue data to CSV files
commit_df.to_csv('kubernetes_commits.csv', index=False)
issue_df.to_csv('kubernetes_issues.csv', index=False)
print('downloaded')


GitCommandError: Cmd('git') failed due to: exit code(128)
  cmdline: git clone -v -- https://github.com/kubernetes/kubernetes.git C:\Users\shahr\AppData\Local\Temp\tmpil7g10zf\kubernetes
  stderr: 'Cloning into 'C:\Users\shahr\AppData\Local\Temp\tmpil7g10zf\kubernetes'...
POST git-upload-pack (185 bytes)
POST git-upload-pack (gzip 55527 to 27929 bytes)
error: RPC failed; curl 92 HTTP/2 stream 0 was not closed cleanly: CANCEL (err 8)
error: 23454 bytes of body are still expected
fetch-pack: unexpected disconnect while reading sideband packet
fatal: early EOF
fatal: fetch-pack: invalid index-pack output
'